In [1]:
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd

cwd = Path.cwd()
data_path = cwd.parent / 'data' / 'clean'
raw_df = pd.read_parquet(data_path / 'btcusdt.parquet')

### TRADINGVIEW INDICATOR
- encode_pattern : Convert positional conditions to binary pattern code
- decode_pattern : Convert binary pattern code back to conditional statements
- Keep statistics of all patterns found and the what targets were hit

In [2]:
# CLASSES
class Signal:
    """
    A class to represent a trading signal.

    Attributes:
    - pattern (str): The pattern code.
    - target (int): The target count.
    - success_rate (float): The success rate of the pattern.
    """

    def __init__(self, pattern: str, target: int, success_rate: float):
        """
        Constructs all the necessary attributes for the Signal object.

        Parameters:
        - pattern (str): The pattern code.
        - target (int): The target count.
        - success_rate (float): The success rate of the pattern.
        """
        self.pattern = pattern
        self.target = target
        self.success_rate = success_rate
    
    def __repr__(self):
        return f"Signal(pattern={self.pattern}, target={self.target}, success_rate={self.success_rate})"
    
    def __gt__(self, other):
        return int(self.success_rate) > int(other.success_rate)
    
    def __lt__(self, other):
        return int(self.success_rate) < int(other.success_rate)
    
    def __eq__(self, other):
        return int(self.success_rate) == int(other.success_rate)


In [3]:
# FUNCTIONS

def resample_data(data: pd.DataFrame, timeframe: str, datetime_col: str = None) -> pd.DataFrame:
    """
    Resamples 1-minute OHLC data into a given timeframe.
    
    Parameters:
    - data (pd.DataFrame): A DataFrame containing 1-minute OHLC data with a DatetimeIndex or a datetime column.
                           The columns should be ['open', 'high', 'low', 'close'].
    - timeframe (str): The resampling timeframe, e.g., '5T', '15T', '30T', '1H', '2H', '4H', '8H', '1D', '1W'.
    - datetime_col (str): The name of the column to use as the datetime index if the index is not a DatetimeIndex.
    
    Returns:
    - resampled_data (pd.DataFrame): A DataFrame with the resampled OHLC data.
    """
    # Create a copy of the data to avoid modifying the original DataFrame
    data = data.copy()

    # Ensure the timeframe is in uppercase
    timeframe = timeframe.lower()
    
    # Check if the DataFrame has the correct columns
    required_columns = ['open', 'high', 'low', 'close']
    if not all(col in data.columns for col in required_columns):
        raise ValueError(f"Input data must contain the following columns: {required_columns}")
    
    # If datetime_col is provided, set it as the index
    if datetime_col:
        if datetime_col not in data.columns:
            raise ValueError(f"The specified datetime column '{datetime_col}' is not in the DataFrame.")
        data[datetime_col] = pd.to_datetime(data[datetime_col])
        data.set_index(datetime_col, inplace=True)
    
    # Check if the index is a DatetimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        raise TypeError("Index of the DataFrame must be a DatetimeIndex or a valid datetime column must be provided.")
    
    # Define the aggregation dictionary for resampling
    ohlc_dict = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    }
    
    # Perform the resampling
    resampled_data = data.resample(timeframe).apply(ohlc_dict).dropna(how='any')
    resampled_data.reset_index(inplace=True, names=[datetime_col])
    
    return resampled_data

def compare_values(value_1: float, value_2: float) -> str:
    """
    Compares two float values and returns '1' if the first value is greater, else returns '0'.
    
    Parameters:
    - value_1 (float): The first value to compare.
    - value_2 (float): The second value to compare.
    
    Returns:
    - (str): '1' if value_1 is greater than value_2, otherwise '0'.
    """
    if value_1 > value_2:
        return '1'
    else:
        return '0'

def filled(price: float, high: float, low: float) -> bool:
    """
    Checks if a price is within the high and low range.

    Parameters:
    - price (float): The price to check.
    - high (float): The high range value.
    - low (float): The low range value.

    Returns:
    - (bool): True if price is within the range, False otherwise.
    """
    return (price <= high) and (price >= low)

def find_patterns(df: pd.DataFrame, hold_period: int, oos_date_start: str, extra_targets: bool=True) -> pd.DataFrame:
    """
    Identifies and counts specific patterns in OHLC data over a given holding period.

    Parameters:
    - df (pd.DataFrame): DataFrame containing OHLC data with columns ['time', 'open', 'high', 'low', 'close'].
    - hold_period (int): The holding period to calculate rolling highest and lowest prices.
    - oos_date_start (str): The out-of-sample start date to filter the DataFrame.
    - extra_targets (bool): Whether to include extra targets in the output DataFrame.

    Returns:
    - pattern_labels (list): List of pattern labels for each row in the DataFrame.
    - count_totals (dict): Dictionary with counts of each pattern found.
    - count_success (dict): Dictionary with success counts for each pattern in different categories.
    """
    # Preprocess the data
    df = df.copy()

    # Calculate rolling highest and lowest prices
    df['highest'] = df['high'].rolling(hold_period).max()
    df['lowest'] = df['low'].rolling(hold_period).min()

    # Store patterns and their statistics
    count_totals = {}
    count_success = {}
    pattern_labels = np.zeros(len(df), dtype=object)

    # Iterate through the DataFrame
    for i in range(hold_period + 1, len(df)):

        bar_0 = df.iloc[i - hold_period]
        bar_1 = df.iloc[i - hold_period - 1]

        _highest = df['highest'].iloc[i]
        _lowest = df['lowest'].iloc[i]

        # Generate pattern string
        _pattern = compare_values(bar_0['high'], bar_1['high']) + \
                   compare_values(bar_0['low'], bar_1['low']) + \
                   compare_values(bar_0['open'], bar_1['open']) + \
                   compare_values(bar_0['close'], bar_1['close']) + \
                   compare_values(bar_0['open'], bar_1['close']) + \
                   compare_values(bar_0['close'], bar_1['open'])
        
        # Add the pattern to the list of labels
        pattern_labels[i] = _pattern

        # Filter DataFrame for in-sample data
        if df.loc[i, 'time'] >= pd.to_datetime(oos_date_start):
            continue
        
        # Initialize pattern in the dictionaries if not present
        if _pattern not in count_totals:
            count_totals[_pattern] = 0
            count_success[_pattern] = {
                'open': 0,
                'high': 0,
                'low': 0,
            }

        # Update the count of the pattern
        count_totals[_pattern] += 1
        
        # Check and update success counts
        if filled(bar_0['open'], _highest, _lowest):
            count_success[_pattern]['open'] += 1
        if filled(bar_0['high'], _highest, _lowest):
            count_success[_pattern]['high'] += 1
        if filled(bar_0['low'], _highest, _lowest):
            count_success[_pattern]['low'] += 1
        
        if extra_targets:
            if filled(bar_1['open'], _highest, _lowest):
                count_success[_pattern]['open_1'] = count_success[_pattern].get('open_1', 0) + 1
            if filled(bar_1['high'], _highest, _lowest):
                count_success[_pattern]['high_1'] = count_success[_pattern].get('high_1', 0) + 1
            if filled(bar_1['low'], _highest, _lowest):
                count_success[_pattern]['low_1'] = count_success[_pattern].get('low_1', 0) + 1
            if filled(bar_1['close'], _highest, _lowest):
                count_success[_pattern]['close_1'] = count_success[_pattern].get('close_1', 0) + 1

    return pattern_labels, count_totals, count_success

def select_patterns(count_totals: Dict[str, int], count_success: Dict[str, Dict[str, int]], 
                    success_rate_threshold: float = 0.75, total_count_threshold: int = 100) -> List[Signal]:
    """
    Selects pattern codes and targets that achieve over a specified success rate and total count.

    Parameters:
    - count_totals (Dict[str, int]): Dictionary with total counts of each pattern.
    - count_success (Dict[str, Dict[str, int]]): Dictionary with success counts for each pattern in different categories.
    - success_rate_threshold (float): The minimum success rate required to select a pattern. Default is 0.80.
    - total_count_threshold (int): The minimum total count required to select a pattern. Default is 100.

    Returns:
    - selected_patterns (List[Signal]): A list of Signal namedtuples containing selected patterns with their success rates.
    """
    if not (set(count_totals.keys()) == set(count_success.keys())):
        raise ValueError("Keys in count_totals and count_success do not match.")
    
    patterns = []

    for pattern in count_totals.keys():
        if count_totals[pattern] > total_count_threshold:
            for target in count_success[pattern].keys():
                success_rate = count_success[pattern][target] / count_totals[pattern]
                if success_rate > success_rate_threshold:
                    signal = Signal(
                        pattern=pattern,
                        target=target,
                        success_rate=round(success_rate, 3)
                    )
                    patterns.append(signal)
    
    # Sort the selected patterns by success rate in descending order
    patterns.sort(reverse=True)

    # In case of two signals with the same pattern code, select the signal with the higher success rate
    # Create a dictionary to store the maximum success rate for each pattern code
    max_success_rate = []
    selected_patterns = []

    for signal in patterns:
        if signal.pattern not in max_success_rate:
            max_success_rate.append(signal.pattern)
            selected_patterns.append(signal)
    
    return selected_patterns

def generate_target_price(df: pd.DataFrame, selected_patterns: List[Signal], pattern_labels: List[int], hold_period:int=1) -> pd.DataFrame:
    """
    Generates target price columns for selected patterns in the DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing OHLC data.
    - selected_patterns (List[Signal]): A list of selected Signal objects.
    - pattern_labels (List[int]): A list of pattern labels corresponding to each row in the DataFrame.

    Returns:
    - pd.DataFrame: The DataFrame with additional columns for target prices.
    """
    # Create a dictionary for selected patterns with their target columns
    selected_patterns_codes = {signal.pattern: signal.target for signal in selected_patterns}

    # Make a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    df['patterns'] = pattern_labels.astype(str)
    df['selected'] = df['patterns'].isin(selected_patterns_codes.keys())
    df['targets_str'] = ''
    
    # Create a shifted DataFrame for previous values
    df_shift = df.shift(1)

    # Assign target columns based on selected patterns
    for i in df[df['selected']].index:
        df.loc[i, 'targets_str'] = selected_patterns_codes[df.loc[i, 'patterns']]

    # Define the conditions and corresponding values for target price selection
    select_conditions = [
        (df['targets_str'] == 'open'),
        (df['targets_str'] == 'high'),
        (df['targets_str'] == 'low'),
        (df['targets_str'] == 'close'),
        (df['targets_str'] == 'open_1'),
        (df['targets_str'] == 'high_1'),
        (df['targets_str'] == 'low_1'),
        (df['targets_str'] == 'close_1')
    ]
    select_values = [
        df['open'], df['high'], df['low'], df['close'], 
        df_shift['open'], df_shift['high'], df_shift['low'], df_shift['close']
    ]

    # Generate the 'targets' column based on conditions and values
    df['targets'] = np.select(select_conditions, select_values, default=np.nan)

    # Forward fill the targets for the holding period
    df['targets'] = df['targets'].ffill(limit=hold_period)
    
    return df

In [4]:
# PARAMETERS
source_timeframe = "4H"
oos_date_start = "2023-01-01"
hold_period = 1

In [5]:
# CODE
df = resample_data(raw_df, source_timeframe, 'time')

# Find patterns
pattern_labels, count_totals, count_success = find_patterns(df, hold_period, oos_date_start)

In [6]:
# Find patterns that achieve over 70% success rate, and the total count is more than 50
selected_patterns = select_patterns(count_totals, count_success)

In [7]:
selected_patterns, len(selected_patterns)

([Signal(pattern=011101, target=open, success_rate=0.786),
  Signal(pattern=110100, target=open_1, success_rate=0.87),
  Signal(pattern=001001, target=open_1, success_rate=0.789),
  Signal(pattern=010000, target=open, success_rate=0.755),
  Signal(pattern=111000, target=low, success_rate=0.769),
  Signal(pattern=011111, target=open, success_rate=0.804),
  Signal(pattern=010101, target=high, success_rate=0.764),
  Signal(pattern=000101, target=high, success_rate=0.837),
  Signal(pattern=011000, target=open_1, success_rate=0.81)],
 9)

### INSPECTOR
- generate filtered pattern labels; including only selected patterns
- generate target_price columns

In [8]:
# Generate the target price columns based on the selected patterns
dff = generate_target_price(df, selected_patterns, pattern_labels)

In [10]:
dff.head()

,time,open,high,low,close,patterns,selected,targets_str,targets
0,2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,0,False,,NaN
1,2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,0,False,,NaN
2,2017-08-17 12:00:00,4427.30,4485.39,4333.42,4352.34,111101,False,,NaN
3,2017-08-17 16:00:00,4352.34,4354.84,4200.74,4325.23,011001,False,,NaN
4,2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,000000,False,,NaN


In [11]:
dates_list = []
for i in dff[dff['selected']].index:
    start_index = i + 1
    end_index = i + hold_period

    if start_index > len(dff) - 1:
        break

    dates_list.append((dff.loc[start_index, 'time'], dff.loc[end_index, 'time'], dff.loc[i, 'targets']))

In [1]:
full_list = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'BCC/USDT', 'NEO/USDT', 'LTC/USDT', 'QTUM/USDT', 'ADA/USDT', 'XRP/USDT', 'EOS/USDT', 'TUSD/USDT', 'IOTA/USDT', 'XLM/USDT', 'ONT/USDT', 'TRX/USDT', 'ETC/USDT', 'ICX/USDT', 'VEN/USDT', 'NULS/USDT', 'VET/USDT', 'PAX/USDT', 'BCHABC/USDT', 'BSV/USDT', 'USDC/USDT', 'LINK/USDT', 'WAVES/USDT', 'BTT/USDT', 'USDS/USDT', 'ONG/USDT', 'HOT/USDT', 'ZIL/USDT', 'ZRX/USDT', 'FET/USDT', 'BAT/USDT', 'XMR/USDT', 'ZEC/USDT', 'IOST/USDT', 'CELR/USDT', 'DASH/USDT', 'NANO/USDT', 'OMG/USDT', 'THETA/USDT', 'ENJ/USDT', 'MITH/USDT', 'MATIC/USDT', 'ATOM/USDT', 'TFUEL/USDT', 'ONE/USDT', 'FTM/USDT', 'ALGO/USDT', 'USDSB/USDT', 'GTO/USDT', 'ERD/USDT', 'DOGE/USDT', 'DUSK/USDT', 'ANKR/USDT', 'WIN/USDT', 'COS/USDT', 'NPXS/USDT', 'COCOS/USDT', 'MTL/USDT', 'TOMO/USDT', 'PERL/USDT', 'DENT/USDT', 'MFT/USDT', 'KEY/USDT', 'STORM/USDT', 'DOCK/USDT', 'WAN/USDT', 'FUN/USDT', 'CVC/USDT', 'CHZ/USDT', 'BAND/USDT', 'BUSD/USDT', 'BEAM/USDT', 'XTZ/USDT', 'REN/USDT', 'RVN/USDT', 'HC/USDT', 'HBAR/USDT', 'NKN/USDT', 'STX/USDT', 'KAVA/USDT', 'ARPA/USDT', 'IOTX/USDT', 'RLC/USDT', 'MCO/USDT', 'CTXC/USDT', 'BCH/USDT', 'TROY/USDT', 'VITE/USDT', 'FTT/USDT', 'EUR/USDT', 'OGN/USDT', 'DREP/USDT', 'BULL/USDT', 'BEAR/USDT', 'ETHBULL/USDT', 'ETHBEAR/USDT', 'TCT/USDT', 'WRX/USDT', 'BTS/USDT', 'LSK/USDT', 'BNT/USDT', 'LTO/USDT', 'EOSBULL/USDT', 'EOSBEAR/USDT', 'XRPBULL/USDT', 'XRPBEAR/USDT', 'STRAT/USDT', 'AION/USDT', 'MBL/USDT', 'COTI/USDT', 'BNBBULL/USDT', 'BNBBEAR/USDT', 'STPT/USDT', 'WTC/USDT', 'DATA/USDT', 'XZC/USDT', 'SOL/USDT', 'CTSI/USDT', 'HIVE/USDT', 'CHR/USDT', 'BTCUP/USDT', 'BTCDOWN/USDT', 'GXS/USDT', 'ARDR/USDT', 'LEND/USDT', 'MDT/USDT', 'STMX/USDT', 'KNC/USDT', 'REP/USDT', 'LRC/USDT', 'PNT/USDT', 'COMP/USDT', 'BKRW/USDT', 'SC/USDT', 'ZEN/USDT', 'SNX/USDT', 'ETHUP/USDT', 'ETHDOWN/USDT', 'ADAUP/USDT', 'ADADOWN/USDT', 'LINKUP/USDT', 'LINKDOWN/USDT', 'VTHO/USDT', 'DGB/USDT', 'GBP/USDT', 'SXP/USDT', 'MKR/USDT', 'DAI/USDT', 'DCR/USDT', 'STORJ/USDT', 'BNBUP/USDT', 'BNBDOWN/USDT', 'XTZUP/USDT', 'XTZDOWN/USDT', 'MANA/USDT', 'AUD/USDT', 'YFI/USDT', 'BAL/USDT', 'BLZ/USDT', 'IRIS/USDT', 'KMD/USDT', 'JST/USDT', 'SRM/USDT', 'ANT/USDT', 'CRV/USDT', 'SAND/USDT', 'OCEAN/USDT', 'NMR/USDT', 'DOT/USDT', 'LUNA/USDT', 'RSR/USDT', 'PAXG/USDT', 'WNXM/USDT', 'TRB/USDT', 'BZRX/USDT', 'SUSHI/USDT', 'YFII/USDT', 'KSM/USDT', 'EGLD/USDT', 'DIA/USDT', 'RUNE/USDT', 'FIO/USDT', 'UMA/USDT', 'EOSUP/USDT', 'EOSDOWN/USDT', 'TRXUP/USDT', 'TRXDOWN/USDT', 'XRPUP/USDT', 'XRPDOWN/USDT', 'DOTUP/USDT', 'DOTDOWN/USDT', 'BEL/USDT', 'WING/USDT', 'LTCUP/USDT', 'LTCDOWN/USDT', 'UNI/USDT', 'NBS/USDT', 'OXT/USDT', 'SUN/USDT', 'AVAX/USDT', 'HNT/USDT', 'FLM/USDT', 'UNIUP/USDT', 'UNIDOWN/USDT', 'ORN/USDT', 'UTK/USDT', 'XVS/USDT', 'ALPHA/USDT', 'AAVE/USDT', 'NEAR/USDT', 'SXPUP/USDT', 'SXPDOWN/USDT', 'FIL/USDT', 'FILUP/USDT', 'FILDOWN/USDT', 'YFIUP/USDT', 'YFIDOWN/USDT', 'INJ/USDT', 'AUDIO/USDT', 'CTK/USDT', 'BCHUP/USDT', 'BCHDOWN/USDT', 'AKRO/USDT', 'AXS/USDT', 'HARD/USDT', 'DNT/USDT', 'STRAX/USDT', 'UNFI/USDT', 'ROSE/USDT', 'AVA/USDT', 'XEM/USDT', 'AAVEUP/USDT', 'AAVEDOWN/USDT', 'SKL/USDT', 'SUSD/USDT', 'SUSHIUP/USDT', 'SUSHIDOWN/USDT', 'XLMUP/USDT', 'XLMDOWN/USDT', 'GRT/USDT', 'JUV/USDT', 'PSG/USDT', '1INCH/USDT', 'REEF/USDT', 'OG/USDT', 'ATM/USDT', 'ASR/USDT', 'CELO/USDT', 'RIF/USDT', 'BTCST/USDT', 'TRU/USDT', 'CKB/USDT', 'TWT/USDT', 'FIRO/USDT', 'LIT/USDT', 'SFP/USDT', 'DODO/USDT', 'CAKE/USDT', 'ACM/USDT', 'BADGER/USDT', 'FIS/USDT', 'OM/USDT', 'POND/USDT', 'DEGO/USDT', 'ALICE/USDT', 'LINA/USDT', 'PERP/USDT', 'RAMP/USDT', 'SUPER/USDT', 'CFX/USDT', 'EPS/USDT', 'AUTO/USDT', 'TKO/USDT', 'PUNDIX/USDT', 'TLM/USDT', '1INCHUP/USDT', '1INCHDOWN/USDT', 'BTG/USDT', 'MIR/USDT', 'BAR/USDT', 'FORTH/USDT', 'BAKE/USDT', 'BURGER/USDT', 'SLP/USDT', 'SHIB/USDT', 'ICP/USDT', 'AR/USDT', 'POLS/USDT', 'MDX/USDT', 'MASK/USDT', 'LPT/USDT', 'NU/USDT', 'XVG/USDT', 'ATA/USDT', 'GTC/USDT', 'TORN/USDT', 'KEEP/USDT', 'ERN/USDT', 'KLAY/USDT', 'PHA/USDT', 'BOND/USDT', 'MLN/USDT', 'DEXE/USDT', 'C98/USDT', 'CLV/USDT', 'QNT/USDT', 'FLOW/USDT', 'TVK/USDT', 'MINA/USDT', 'RAY/USDT', 'FARM/USDT', 'ALPACA/USDT', 'QUICK/USDT', 'MBOX/USDT', 'FOR/USDT', 'REQ/USDT', 'GHST/USDT', 'WAXP/USDT', 'TRIBE/USDT', 'GNO/USDT', 'XEC/USDT', 'ELF/USDT', 'DYDX/USDT', 'POLY/USDT', 'IDEX/USDT', 'VIDT/USDT', 'USDP/USDT', 'GALA/USDT', 'ILV/USDT', 'YGG/USDT', 'SYS/USDT', 'DF/USDT', 'FIDA/USDT', 'FRONT/USDT', 'CVP/USDT', 'AGLD/USDT', 'RAD/USDT', 'BETA/USDT', 'RARE/USDT', 'LAZIO/USDT', 'CHESS/USDT', 'ADX/USDT', 'AUCTION/USDT', 'DAR/USDT', 'BNX/USDT', 'RGT/USDT', 'MOVR/USDT', 'CITY/USDT', 'ENS/USDT', 'KP3R/USDT', 'QI/USDT', 'PORTO/USDT', 'POWR/USDT', 'VGX/USDT', 'JASMY/USDT', 'AMP/USDT', 'PLA/USDT', 'PYR/USDT', 'RNDR/USDT', 'ALCX/USDT', 'SANTOS/USDT', 'MC/USDT', 'ANY/USDT', 'BICO/USDT', 'FLUX/USDT', 'FXS/USDT', 'VOXEL/USDT', 'HIGH/USDT', 'CVX/USDT', 'PEOPLE/USDT', 'OOKI/USDT', 'SPELL/USDT', 'UST/USDT', 'JOE/USDT', 'ACH/USDT', 'IMX/USDT', 'GLMR/USDT', 'LOKA/USDT', 'SCRT/USDT', 'API3/USDT', 'BTTC/USDT', 'ACA/USDT', 'ANC/USDT', 'XNO/USDT', 'WOO/USDT', 'ALPINE/USDT', 'T/USDT', 'ASTR/USDT', 'GMT/USDT', 'KDA/USDT', 'APE/USDT', 'BSW/USDT', 'BIFI/USDT', 'MULTI/USDT', 'STEEM/USDT', 'MOB/USDT', 'NEXO/USDT', 'REI/USDT', 'GAL/USDT', 'LDO/USDT', 'EPX/USDT', 'OP/USDT', 'LEVER/USDT', 'STG/USDT', 'LUNC/USDT', 'GMX/USDT', 'NEBL/USDT', 'POLYX/USDT', 'APT/USDT', 'OSMO/USDT', 'HFT/USDT', 'PHB/USDT', 'HOOK/USDT', 'MAGIC/USDT', 'HIFI/USDT', 'RPL/USDT', 'PROS/USDT', 'AGIX/USDT', 'GNS/USDT', 'SYN/USDT', 'VIB/USDT', 'SSV/USDT', 'LQTY/USDT', 'AMB/USDT', 'BETH/USDT', 'USTC/USDT', 'GAS/USDT', 'GLM/USDT', 'PROM/USDT', 'QKC/USDT', 'UFT/USDT', 'ID/USDT', 'ARB/USDT', 'LOOM/USDT', 'OAX/USDT', 'RDNT/USDT', 'WBTC/USDT', 'EDU/USDT', 'SUI/USDT', 'AERGO/USDT', 'PEPE/USDT', 'FLOKI/USDT', 'AST/USDT', 'SNT/USDT', 'COMBO/USDT', 'MAV/USDT', 'PENDLE/USDT', 'ARKM/USDT', 'WBETH/USDT', 'WLD/USDT', 'FDUSD/USDT', 'SEI/USDT', 'CYBER/USDT', 'ARK/USDT', 'CREAM/USDT', 'GFT/USDT', 'IQ/USDT', 'NTRN/USDT', 'TIA/USDT', 'MEME/USDT', 'ORDI/USDT', 'BEAMX/USDT', 'PIVX/USDT', 'VIC/USDT', 'BLUR/USDT', 'VANRY/USDT', 'AEUR/USDT', 'JTO/USDT', '1000SATS/USDT', 'BONK/USDT', 'ACE/USDT', 'NFP/USDT', 'AI/USDT', 'XAI/USDT', 'MANTA/USDT', 'ALT/USDT', 'JUP/USDT', 'PYTH/USDT', 'RONIN/USDT', 'DYM/USDT', 'PIXEL/USDT', 'STRK/USDT', 'PORTAL/USDT', 'PDA/USDT', 'AXL/USDT', 'WIF/USDT', 'METIS/USDT', 'AEVO/USDT', 'BOME/USDT', 'ETHFI/USDT', 'ENA/USDT', 'W/USDT', 'TNSR/USDT', 'SAGA/USDT', 'TAO/USDT', 'OMNI/USDT', 'REZ/USDT', 'BB/USDT', 'NOT/USDT']

In [2]:
old_pairs_str = "BINANCE:1000BONKUSDT.P,BINANCE:1000FLOKIUSDT.P,BINANCE:1000LUNCUSDT.P,BINANCE:1000PEPEUSDT.P,BINANCE:1000RATSUSDT.P,BINANCE:1000SATSUSDT,BINANCE:1000SATSUSDT.P,BINANCE:1000SHIBUSDT.P,BINANCE:1000XECUSDT.P,BINANCE:1INCHUSDT,BINANCE:1INCHUSDT.P,BINANCE:ACAUSDT,BINANCE:ACEUSDT.P,BINANCE:ACHUSDT,BINANCE:ACHUSDT.P,BINANCE:ADAUSDT,BINANCE:ADAUSDT.P,BINANCE:ADXUSDT,BINANCE:AERGOUSDT,BINANCE:AEVOUSDT,BINANCE:AEVOUSDT.P,BINANCE:AGIXUSDT,BINANCE:AGIXUSDT.P,BINANCE:AGLDUSDT,BINANCE:AGLDUSDT.P,BINANCE:AIUSDT,BINANCE:AIUSDT.P,BINANCE:AKROUSDT,BINANCE:ALGOUSDT,BINANCE:ALGOUSDT.P,BINANCE:ALICEUSDT.P,BINANCE:ALPACAUSDT,BINANCE:ALPHAUSDT,BINANCE:ALPHAUSDT.P,BINANCE:ALTUSDT,BINANCE:ALTUSDT.P,BINANCE:AMBUSDT,BINANCE:AMBUSDT.P,BINANCE:AMPUSDT,BINANCE:ANKRUSDT,BINANCE:ANKRUSDT.P,BINANCE:APEUSDT,BINANCE:APEUSDT.P,BINANCE:API3USDT.P,BINANCE:APTUSDT,BINANCE:APTUSDT.P,BINANCE:ARBUSDT,BINANCE:ARBUSDT.P,BINANCE:ARDRUSDT,BINANCE:ARKMUSDT,BINANCE:ARKMUSDT.P,BINANCE:ARKUSDT.P,BINANCE:ARPAUSDT,BINANCE:ARPAUSDT.P,BINANCE:ASTRUSDT,BINANCE:ASTRUSDT.P,BINANCE:ASTUSDT,BINANCE:ATAUSDT,BINANCE:ATAUSDT.P,BINANCE:ATOMUSDT.P,BINANCE:AUDIOUSDT,BINANCE:AVAXUSDT.P,BINANCE:AXLUSDT,BINANCE:AXLUSDT.P,BINANCE:AXSUSDT.P,BINANCE:BAKEUSDT,BINANCE:BAKEUSDT.P,BINANCE:BANDUSDT.P,BINANCE:BATUSDT,BINANCE:BATUSDT.P,BINANCE:BBUSDT,BINANCE:BBUSDT.P,BINANCE:BEAMXUSDT,BINANCE:BEAMXUSDT.P,BINANCE:BELUSDT,BINANCE:BELUSDT.P,BINANCE:BETAUSDT,BINANCE:BICOUSDT,BINANCE:BICOUSDT.P,BINANCE:BIGTIMEUSDT.P,BINANCE:BLURUSDT,BINANCE:BLURUSDT.P,BINANCE:BLZUSDT,BINANCE:BLZUSDT.P,BINANCE:BNTUSDT.P,BINANCE:BNXUSDT,BINANCE:BNXUSDT.P,BINANCE:BOMEUSDT,BINANCE:BOMEUSDT.P,BINANCE:BONKUSDT,BINANCE:BSWUSDT,BINANCE:BTTCUSDT,BINANCE:C98USDT,BINANCE:C98USDT.P,BINANCE:CAKEUSDT,BINANCE:CAKEUSDT.P,BINANCE:CELOUSDT,BINANCE:CELOUSDT.P,BINANCE:CELRUSDT,BINANCE:CELRUSDT.P,BINANCE:CFXUSDT,BINANCE:CFXUSDT.P,BINANCE:CHESSUSDT,BINANCE:CHRUSDT,BINANCE:CHRUSDT.P,BINANCE:CHZUSDT,BINANCE:CHZUSDT.P,BINANCE:CKBUSDT,BINANCE:CKBUSDT.P,BINANCE:CLVUSDT,BINANCE:COMBOUSDT.P,BINANCE:COSUSDT,BINANCE:COTIUSDT,BINANCE:COTIUSDT.P,BINANCE:CRVUSDT,BINANCE:CRVUSDT.P,BINANCE:CTSIUSDT,BINANCE:CTSIUSDT.P,BINANCE:CTXCUSDT,BINANCE:CVCUSDT,BINANCE:CYBERUSDT.P,BINANCE:DARUSDT,BINANCE:DARUSDT.P,BINANCE:DATAUSDT,BINANCE:DENTUSDT,BINANCE:DENTUSDT.P,BINANCE:DFUSDT,BINANCE:DGBUSDT,BINANCE:DOCKUSDT,BINANCE:DODOUSDT,BINANCE:DODOXUSDT.P,BINANCE:DOGEUSDT,BINANCE:DOGEUSDT.P,BINANCE:DOTUSDT.P,BINANCE:DUSKUSDT,BINANCE:DUSKUSDT.P,BINANCE:DYDXUSDT,BINANCE:DYDXUSDT.P,BINANCE:DYMUSDT.P,BINANCE:EDUUSDT,BINANCE:EDUUSDT.P,BINANCE:ENAUSDT,BINANCE:ENAUSDT.P,BINANCE:ENJUSDT,BINANCE:ENJUSDT.P,BINANCE:ENSUSDT.P,BINANCE:EOSUSDT,BINANCE:EOSUSDT.P,BINANCE:EPXUSDT,BINANCE:ETCUSDT.P,BINANCE:ETHFIUSDT,BINANCE:ETHFIUSDT.P,BINANCE:EURUSDT,BINANCE:FDUSDTRY,BINANCE:FDUSDUSDT,BINANCE:FETUSDT,BINANCE:FETUSDT.P,BINANCE:FIDAUSDT,BINANCE:FILUSDT,BINANCE:FILUSDT.P,BINANCE:FIOUSDT,BINANCE:FLMUSDT,BINANCE:FLMUSDT.P,BINANCE:FLOKIUSDT,BINANCE:FLOWUSDT,BINANCE:FLOWUSDT.P,BINANCE:FORUSDT,BINANCE:FRONTUSDT,BINANCE:FRONTUSDT.P,BINANCE:FTMUSDT,BINANCE:FTMUSDT.P,BINANCE:FUNUSDT,BINANCE:FXSUSDT,BINANCE:FXSUSDT.P,BINANCE:GALAUSDT,BINANCE:GALAUSDT.P,BINANCE:GALUSDT.P,BINANCE:GASUSDT.P,BINANCE:GFTUSDT,BINANCE:GLMRUSDT,BINANCE:GLMUSDT,BINANCE:GLMUSDT.P,BINANCE:GMTUSDT,BINANCE:GMTUSDT.P,BINANCE:GRTUSDT,BINANCE:GRTUSDT.P,BINANCE:GTCUSDT.P,BINANCE:HARDUSDT,BINANCE:HBARUSDT,BINANCE:HBARUSDT.P,BINANCE:HFTUSDT,BINANCE:HFTUSDT.P,BINANCE:HIFIUSDT,BINANCE:HIFIUSDT.P,BINANCE:HIGHUSDT.P,BINANCE:HOOKUSDT,BINANCE:HOOKUSDT.P,BINANCE:HOTUSDT,BINANCE:HOTUSDT.P,BINANCE:ICPUSDT.P,BINANCE:ICXUSDT,BINANCE:ICXUSDT.P,BINANCE:IDEXUSDT,BINANCE:IDUSDT,BINANCE:IDUSDT.P,BINANCE:IMXUSDT,BINANCE:IMXUSDT.P,BINANCE:INJUSDT.P,BINANCE:IOSTUSDT,BINANCE:IOSTUSDT.P,BINANCE:IOTAUSDT,BINANCE:IOTAUSDT.P,BINANCE:IOTXUSDT,BINANCE:IOTXUSDT.P,BINANCE:IQUSDT,BINANCE:IRISUSDT,BINANCE:JASMYUSDT,BINANCE:JASMYUSDT.P,BINANCE:JOEUSDT,BINANCE:JOEUSDT.P,BINANCE:JSTUSDT,BINANCE:JTOUSDT,BINANCE:JTOUSDT.P,BINANCE:JUPUSDT,BINANCE:JUPUSDT.P,BINANCE:KASUSDT.P,BINANCE:KAVAUSDT,BINANCE:KAVAUSDT.P,BINANCE:KEYUSDT,BINANCE:KEYUSDT.P,BINANCE:KLAYUSDT,BINANCE:KLAYUSDT.P,BINANCE:KMDUSDT,BINANCE:KNCUSDT,BINANCE:KNCUSDT.P,BINANCE:LDOUSDT,BINANCE:LDOUSDT.P,BINANCE:LEVERUSDT,BINANCE:LEVERUSDT.P,BINANCE:LINAUSDT,BINANCE:LINAUSDT.P,BINANCE:LINKUSDT,BINANCE:LINKUSDT.P,BINANCE:LITUSDT.P,BINANCE:LOKAUSDT,BINANCE:LOOMUSDT,BINANCE:LOOMUSDT.P,BINANCE:LPTUSDT.P,BINANCE:LQTYUSDT,BINANCE:LQTYUSDT.P,BINANCE:LRCUSDT,BINANCE:LRCUSDT.P,BINANCE:LSKUSDT.P,BINANCE:LTOUSDT,BINANCE:LUNA2USDT.P,BINANCE:LUNAUSDT,BINANCE:LUNCUSDT,BINANCE:MAGICUSDT,BINANCE:MAGICUSDT.P,BINANCE:MANAUSDT,BINANCE:MANAUSDT.P,BINANCE:MANTAUSDT,BINANCE:MANTAUSDT.P,BINANCE:MASKUSDT.P,BINANCE:MATICUSDT,BINANCE:MATICUSDT.P,BINANCE:MAVIAUSDT.P,BINANCE:MAVUSDT,BINANCE:MAVUSDT.P,BINANCE:MBLUSDT,BINANCE:MBOXUSDT,BINANCE:MDTUSDT,BINANCE:MDXUSDT,BINANCE:MEMEUSDT,BINANCE:MEMEUSDT.P,BINANCE:MINAUSDT,BINANCE:MINAUSDT.P,BINANCE:MTLUSDT.P,BINANCE:MYROUSDT.P,BINANCE:NEARUSDT,BINANCE:NEARUSDT.P,BINANCE:NFPUSDT,BINANCE:NFPUSDT.P,BINANCE:NKNUSDT,BINANCE:NKNUSDT.P,BINANCE:NOTUSDT,BINANCE:NOTUSDT.P,BINANCE:NTRNUSDT,BINANCE:NTRNUSDT.P,BINANCE:OAXUSDT,BINANCE:OCEANUSDT,BINANCE:OCEANUSDT.P,BINANCE:OGNUSDT,BINANCE:OGNUSDT.P,BINANCE:OMGUSDT,BINANCE:OMGUSDT.P,BINANCE:OMUSDT,BINANCE:OMUSDT.P,BINANCE:ONDOUSDT.P,BINANCE:ONEUSDT,BINANCE:ONEUSDT.P,BINANCE:ONGUSDT,BINANCE:ONGUSDT.P,BINANCE:ONTUSDT,BINANCE:ONTUSDT.P,BINANCE:OOKIUSDT,BINANCE:OPUSDT,BINANCE:OPUSDT.P,BINANCE:ORBSUSDT.P,BINANCE:ORDIUSDT.P,BINANCE:OXTUSDT,BINANCE:OXTUSDT.P,BINANCE:PDAUSDT,BINANCE:PENDLEUSDT,BINANCE:PENDLEUSDT.P,BINANCE:PEOPLEUSDT,BINANCE:PEOPLEUSDT.P,BINANCE:PEPEUSDT,BINANCE:PERPUSDT,BINANCE:PERPUSDT.P,BINANCE:PHAUSDT,BINANCE:PHBUSDT.P,BINANCE:PIXELUSDT,BINANCE:PIXELUSDT.P,BINANCE:POLYXUSDT,BINANCE:POLYXUSDT.P,BINANCE:PONDUSDT,BINANCE:PORTALUSDT,BINANCE:PORTALUSDT.P,BINANCE:POWRUSDT,BINANCE:POWRUSDT.P,BINANCE:PYTHUSDT,BINANCE:PYTHUSDT.P,BINANCE:QIUSDT,BINANCE:QKCUSDT,BINANCE:QUICKUSDT,BINANCE:RADUSDT,BINANCE:RAREUSDT,BINANCE:RDNTUSDT,BINANCE:RDNTUSDT.P,BINANCE:REEFUSDT,BINANCE:REEFUSDT.P,BINANCE:REIUSDT,BINANCE:RENUSDT,BINANCE:RENUSDT.P,BINANCE:REQUSDT,BINANCE:REZUSDT,BINANCE:REZUSDT.P,BINANCE:RIFUSDT,BINANCE:RIFUSDT.P,BINANCE:RNDRUSDT,BINANCE:RNDRUSDT.P,BINANCE:RONINUSDT.P,BINANCE:ROSEUSDT,BINANCE:ROSEUSDT.P,BINANCE:RSRUSDT,BINANCE:RSRUSDT.P,BINANCE:RUNEUSDT,BINANCE:RUNEUSDT.P,BINANCE:RVNUSDT,BINANCE:RVNUSDT.P,BINANCE:SAGAUSDT,BINANCE:SAGAUSDT.P,BINANCE:SANDUSDT,BINANCE:SANDUSDT.P,BINANCE:SCRTUSDT,BINANCE:SCUSDT,BINANCE:SEIUSDT,BINANCE:SEIUSDT.P,BINANCE:SFPUSDT.P,BINANCE:SHIBUSDT,BINANCE:SKLUSDT,BINANCE:SKLUSDT.P,BINANCE:SLPUSDT,BINANCE:SNTUSDT,BINANCE:SNXUSDT,BINANCE:SNXUSDT.P,BINANCE:SOLUSDT.P,BINANCE:SPELLUSDT,BINANCE:SPELLUSDT.P,BINANCE:STEEMUSDT,BINANCE:STEEMUSDT.P,BINANCE:STGUSDT,BINANCE:STGUSDT.P,BINANCE:STMXUSDT,BINANCE:STMXUSDT.P,BINANCE:STORJUSDT,BINANCE:STORJUSDT.P,BINANCE:STPTUSDT,BINANCE:STRAXUSDT,BINANCE:STRKUSDT,BINANCE:STRKUSDT.P,BINANCE:STXUSDT,BINANCE:STXUSDT.P,BINANCE:SUIUSDT,BINANCE:SUIUSDT.P,BINANCE:SUNUSDT,BINANCE:SUPERUSDT.P,BINANCE:SUSHIUSDT,BINANCE:SUSHIUSDT.P,BINANCE:SXPUSDT,BINANCE:SXPUSDT.P,BINANCE:SYNUSDT,BINANCE:SYSUSDT,BINANCE:TFUELUSDT,BINANCE:THETAUSDT,BINANCE:THETAUSDT.P,BINANCE:TIAUSDT.P,BINANCE:TLMUSDT,BINANCE:TLMUSDT.P,BINANCE:TNSRUSDT,BINANCE:TNSRUSDT.P,BINANCE:TOKENUSDT.P,BINANCE:TONUSDT.P,BINANCE:TROYUSDT,BINANCE:TRUUSDT,BINANCE:TRUUSDT.P,BINANCE:TRXUSDT,BINANCE:TRXUSDT.P,BINANCE:TUSDT,BINANCE:TUSDT.P,BINANCE:TWTUSDT.P,BINANCE:UMAUSDT.P,BINANCE:UNFIUSDT.P,BINANCE:UNIUSDT,BINANCE:UNIUSDT.P,BINANCE:USDCUSDT,BINANCE:USDTBRL,BINANCE:USDTDAI,BINANCE:USDTTRY,BINANCE:USTCUSDT,BINANCE:USTCUSDT.P,BINANCE:UTKUSDT,BINANCE:VANRYUSDT,BINANCE:VANRYUSDT.P,BINANCE:VETUSDT,BINANCE:VETUSDT.P,BINANCE:VGXUSDT,BINANCE:VIBUSDT,BINANCE:VICUSDT,BINANCE:VIDTUSDT,BINANCE:VITEUSDT,BINANCE:VOXELUSDT,BINANCE:VTHOUSDT,BINANCE:WANUSDT,BINANCE:WAVESUSDT.P,BINANCE:WAXPUSDT,BINANCE:WAXPUSDT.P,BINANCE:WIFUSDT,BINANCE:WIFUSDT.P,BINANCE:WINUSDT,BINANCE:WLDUSDT,BINANCE:WLDUSDT.P,BINANCE:WOOUSDT,BINANCE:WOOUSDT.P,BINANCE:WRXUSDT,BINANCE:WUSDT,BINANCE:WUSDT.P,BINANCE:XAIUSDT,BINANCE:XAIUSDT.P,BINANCE:XECUSDT,BINANCE:XEMUSDT,BINANCE:XEMUSDT.P,BINANCE:XLMUSDT,BINANCE:XLMUSDT.P,BINANCE:XRPUSDT,BINANCE:XRPUSDT.P,BINANCE:XTZUSDT,BINANCE:XTZUSDT.P,BINANCE:XVGUSDT,BINANCE:XVGUSDT.P,BINANCE:YGGUSDT,BINANCE:YGGUSDT.P,BINANCE:ZETAUSDT.P,BINANCE:ZILUSDT,BINANCE:ZILUSDT.P,BINANCE:ZRXUSDT,BINANCE:ZRXUSDT.P,BINANCE:AVAXUSDT"


'ER'